<a href="https://colab.research.google.com/github/hui509/Titanic_Survival_Prediction/blob/main/%E5%B0%88%E9%A1%8C%E5%AF%A6%E4%BD%9C%EF%BD%9C%E9%90%B5%E9%81%94%E5%B0%BC%E8%99%9F%E5%AD%98%E6%B4%BB%E9%A0%90%E6%B8%AC%E5%88%86%E6%9E%90.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **載入資料**

In [333]:
# 載入訓練集、測試集、示範檔資料
import pandas as pd
train_url = 'https://raw.githubusercontent.com/hui509/Titanic_Survival_Prediction/main/raw_data/train.csv'
test_url = 'https://raw.githubusercontent.com/hui509/Titanic_Survival_Prediction/main/raw_data/test.csv'
submit_url = 'https://raw.githubusercontent.com/hui509/Titanic_Survival_Prediction/main/raw_data/gender_submission.csv'

train = pd.read_csv(train_url)
test = pd.read_csv(test_url)
submit = pd.read_csv(submit_url)

# **認識資料**

1. 訓練集｜891筆資料，12個欄位，包含Survival，其中，Age、Cabin、Embarked有缺失值。

2. 測試集｜418筆資料，11個欄位，缺少Survival，其中，Age、Fare、Cabin有缺失值。

In [ ]:
# 觀察訓練集
train.info()

In [ ]:
# 觀察測試集
test.info()

In [ ]:
# 訓練集缺失值統計
train.isnull().sum()

In [ ]:
# 測試集缺失值統計
test.isnull().sum()

# **合併訓練集與測試集**

In [338]:
# 合併訓練集、測試集，方便後續模型分析
data = train._append(test)
data.reset_index(inplace=True, drop=True)

# **資料探索與分析（EDA）**

1. 罹難者中，多屬於低票價的乘客。
2. Age 中，15歲以下的少年存活機率較高。
3. Sex 中，女性乘客的存活機會高於男性。
4. Pclass 中，一等艙的乘客有較高的存活機會。
5. Embarked 中，從 Cherbourg 登船者的存活機會較高。

In [ ]:
# 使用 Pearson 係數計算相關性
cor = data.corr()

# 使用seaborn和matplotlib，視覺化呈現相關性
import seaborn as sns
import matplotlib.pyplot as plt
plt.figure(figsize=(10,8))
sns.heatmap(cor,annot=True,linewidth=0.5,cmap='coolwarm')
plt.title('Correlation Matrix of Titanic')
plt.show()

In [ ]:
# 定義distribution函數，以KDE圖觀察欄位與Survived之間的關係

def distribution(data,var,target,**kwargs):
  # 設定row和col參數
  row = kwargs.get('row',None)
  col = kwargs.get('col',None)

  # 使用seaborn函數製圖
  facet = sns.FacetGrid(data,row=row,col=col,hue=target,aspect=3)
  facet.map(sns.kdeplot,var,fill=True)
  facet.set(xlim=(0,data[var].max()))
  facet.add_legend()

# 觀察Fare和Survived的關係
distribution(data,var='Fare',target='Survived')
plt.show()

# 觀察Age和Survived的關係
distribution(data,var='Age',target='Survived')
plt.show()

In [ ]:
# 定義categories函數，以長條圖觀察欄位與Survived之間的關係

def categories(data,cat,target,**kwargs):
  # 設定row和col參數
  row = kwargs.get('row',None)
  col = kwargs.get('col',None)

  # 使用seaborn函數製圖
  facet = sns.FacetGrid(data,row=row,col=col,height=4)
  facet.map(sns.barplot,cat,target,color='lightblue')
  facet.add_legend()

# 觀察Sex和Survived的關係
categories(data,cat='Sex',target='Survived')
plt.show()

# 觀察Pclass和Survived的關係
categories(data,cat='Pclass',target='Survived')
plt.show()

# 觀察Embarked和Survived的關係
categories(data,cat='Embarked',target='Survived')
plt.show()

# **資料清理與型態轉換**

1. 缺失值｜Age、Fare 以平均數填補

2. 型態轉換｜Sex、Embarked







In [342]:
# Age 和 fare 為數值型態，以平均數填補
data['Age'].fillna(data['Age'].mean(),inplace=True)
data['Fare'].fillna(data['Fare'].mean(),inplace=True)

In [343]:
# 使用 OneHotEncoder 轉換 Embarked 為數值型態
data = pd.get_dummies(data,columns=['Embarked'],dtype=int)

In [344]:
# 將 Sex 轉換為數值型態
data['Sex'] = data['Sex'].replace({'male':1,'female':0})

# **特徵工程**

1. 船上親屬人數的多寡，是否會影響生存機會？ **親屬人數2-4人存活機率高。**


2. 乘客頭銜隱含社會地位，是否會影響生存機會？ **具有女性頭銜的乘客，存活率較高。**

In [ ]:
#新增FamilySize欄位統整乘客在船上的親屬人數
data['FamilySize'] = data['SibSp']+data['Parch']+1

#使用distribution函數，以KDE圖觀察與Survived之間的關係
distribution(data,var='FamilySize',target='Survived')
plt.show()

# 定義family函數，將親屬人數進行分類
def family(size):
  if size == 1:
    return 'Single'
  elif 2<=size<=4:
    return 'Small'
  else:
    return 'Large'

# 新增Family_Type說明所屬分類：單身、小家庭、大家庭
data['Family_Type'] = data['FamilySize'].map(family)

# 使用OneHotEncoder轉換類別欄位並join數據
Family_ohe = pd.get_dummies(data['Family_Type'], prefix='Family_')
data = data.join(Family_ohe)

In [ ]:
# 定義extraction函數，抽取乘客頭銜
def extraction(name):
  title = name.split(',')[1].split('.')[0].strip()
  return title

# 新增Raw Title欄位放置抽取出的乘客頭銜
data['Raw Title'] = data['Name'].map(extraction)

# 觀察Name中頭銜的種類
title_set = set()
for name in data['Name']:
  title_set.add(extraction(name))

# 製作title對應的dictionary
title_dict = {
'Col':      'Officer',
'Major':     'Officer',
'Capt':      'Officer',
'Jonkheer':    'Royalty',
'Don':      'Royalty',
'Dona':      'Royalty',
'Sir':      'Royalty',
'the Countess': 'Royalty',
'Lady':      'Royalty',
'Dr':       'Royalty',
'Rev':      'Royalty',
'Mr':       'Mr',
'Ms':       'Ms',
'Miss':      'Miss',
'Mlle':      'Miss',
'Mrs':       'Mrs',
'Mme':       'Mrs',
'Master':     'Master'
}

# 新增Title欄位放置經轉換的頭銜
data['Title'] = data['Raw Title'].map(title_dict)

# 使用categories函數，以長條圖觀察Title與Survived的關係
categories(data,cat='Title',target='Survived')
plt.show()

# 使用OneHotEncoder轉換類別欄位並join數據
Title_ohe = pd.get_dummies(data['Title'], prefix='Title_')
data = data.join(Title_ohe)

In [347]:
# 刪除模型無法運算的欄位
data.drop(columns=['PassengerId','Name','Ticket','Cabin','Family_Type','Raw Title','Title'],axis=1,inplace=True)

# **模型分析**

In [348]:
# 載入模型
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score

# 產生訓練集和測試集
Train = data[ pd.notnull(data.Survived) ].copy()
Test = data[ pd.isnull(data.Survived) ].copy()

In [349]:
# X_Train 為訓練集的特徵欄位
# Y_Train 為訓練集的標籤欄位
X_Train = Train.drop( ['Survived'], axis=1 )
Y_Train = Train.Survived

# 測試集刪除 Survived 欄位
Test.drop( ['Survived'], axis=1, inplace=True )

In [350]:
# 使用Logistic Regression分析
model = LogisticRegression(random_state=0,max_iter=3000)
model.fit( X_Train, Y_Train )
scores = cross_val_score(model,X_Train,Y_Train.values.ravel(),cv=5,scoring='accuracy').mean()
print(scores)

0.8237838177138912


# **模型優化**

In [351]:
# 參數調整
from sklearn.model_selection import GridSearchCV
clf = GridSearchCV(model,
    {'C': [0.01, 0.1, 1, 10],'solver': ['liblinear','lbfgs']}, cv=5)
clf.fit( X_Train, Y_Train )

# 取得調整後的各種結果
df = pd.DataFrame(clf.cv_results_)
df[['param_C','param_solver','mean_test_score']].sort_values(by='mean_test_score',ascending = False)
# print(clf.best_score_)
# print(clf.best_params_)

,param_C,param_solver,mean_test_score
6,10,liblinear,0.826018
4,1,liblinear,0.824901
7,10,lbfgs,0.824901
5,1,lbfgs,0.823784
3,0.1,lbfgs,0.818178
2,0.1,liblinear,0.813665
1,0.01,lbfgs,0.782336
0,0.01,liblinear,0.750932


In [352]:
# 使用最佳的參數，再次進行Logistic Regression分析
model = LogisticRegression(C=10,solver='liblinear',random_state=0,max_iter=3000)
model.fit( X_Train, Y_Train )
scores = cross_val_score(model,X_Train,Y_Train.values.ravel(),cv=5,scoring='accuracy').mean()
print(scores)

0.8260184545853996


In [ ]:
# 預測測試集的 Survived 結果
Test_pred = model.predict(Test)
print( Test_pred )

In [ ]:
# 取得提交檔案
submit['Survived'] = Test_pred.astype(int)
submit.to_csv( 'Titanic_LogisticRegression_params.csv', index=False )
print( f'預測結果：' )
print(submit)

In [ ]:
# 將提交檔案存入本機
from google.colab import files
files.download('Titanic_LogisticRegression_params.csv')